In [ ]:
import matplotlib.pyplot as plt
from fno import FNO

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from data_process import DataProcess, split_data, delete_row, mirror_fill, choose_line
from index_cal import mse_c, mae_c, std_c, r2_c, iqr_c
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# ti+2 newly processed

In [3]:
stable = pd.read_csv('stablechange_data.csv', header = None).to_numpy()
random = pd.read_csv('randomchange_data.csv', header = None).to_numpy()

stabledl = pd.read_csv('stablechange_dl.csv', header = None).to_numpy()
randomdl = pd.read_csv('randomchange_dl.csv', header = None).to_numpy()

stablekapa = pd.read_csv('stablechange_kapa.csv', header = None).to_numpy()
randomkapa = pd.read_csv('randomchange_kapa.csv', header = None).to_numpy()

stabletau = pd.read_csv('stablechange_tau.csv', header = None).to_numpy()
randomtau = pd.read_csv('randomchange_tau.csv', header = None).to_numpy()

stablekapaidea = pd.read_csv('stablechange_kapa_idea.csv', header = None).to_numpy()
randomkapaidea = pd.read_csv('randomchange_kapa_idea.csv', header = None).to_numpy()

stabletauidea = pd.read_csv('stablechange_tau_idea.csv', header = None).to_numpy()
randomtauidea = pd.read_csv('randomchange_tau_idea.csv', header = None).to_numpy()

stablealpha = pd.read_csv('stablechange_alpha.csv', header = None).to_numpy()
randomalpha = pd.read_csv('randomchange_alpha.csv', header = None).to_numpy()

stableU = pd.read_csv('stablechange_U.csv', header = None).to_numpy()
randomU = pd.read_csv('randomchange_U.csv', header = None).to_numpy()

stablew = pd.read_csv('stablechange_w.csv', header = None).to_numpy()
randomw = pd.read_csv('randomchange_w.csv', header = None).to_numpy()

stable = torch.from_numpy(stable).float().to(device)
random = torch.from_numpy(random).float().to(device)

stabledl = torch.from_numpy(stabledl).float().to(device)
randomdl = torch.from_numpy(randomdl).float().to(device)

stablekapa = torch.from_numpy(stablekapa).float().to(device)
randomkapa = torch.from_numpy(randomkapa).float().to(device)

stabletau = torch.from_numpy(stabletau).float().to(device)
randomtau = torch.from_numpy(randomtau).float().to(device)

stablekapaidea = torch.from_numpy(stablekapaidea).float().to(device)
randomkapaidea = torch.from_numpy(randomkapaidea).float().to(device)

stabletauidea = torch.from_numpy(stabletauidea).float().to(device)
randomtauidea = torch.from_numpy(randomtauidea).float().to(device)

stablealpha = torch.from_numpy(stablealpha).float().to(device)
randomalpha = torch.from_numpy(randomalpha).float().to(device)

stableU = torch.from_numpy(stableU).float().to(device)
randomU = torch.from_numpy(randomU).float().to(device)

stablew = torch.from_numpy(stablew).float().to(device)
randomw = torch.from_numpy(randomw).float().to(device)


In [4]:
index1 = [0, 1, 2, 3, 4, 5, 12]
index2 = [0, 1, 2, 3, 4, 5, 23]

feature1 = stable[:, index1]
feature2 = random[:, index2]

feature_normalize11 = feature1.repeat_interleave(18, dim = 0)
feature_normalize22 = feature2.repeat_interleave(8, dim = 0)

stabledl = stabledl[:, 1:-1].reshape(-1, 1)
randomdl = randomdl[:, 1:-1].reshape(-1, 1)

stabledl = stabledl
randomdl = randomdl

feature_normalize11 = torch.cat((feature_normalize11, stabledl), dim = 1) # [allsize, 8]
feature_normalize22 = torch.cat((feature_normalize22, randomdl), dim = 1)


In [5]:
num1_s = torch.round(stable[:, 13:14]).int()
num2_s = torch.round(stable[:, 14:15]).int()
local_k1 = torch.ones((stable.shape[0], 1200)).to(device)
local_k2 = torch.ones((stable.shape[0], 1200)).to(device)

for i in range(stable.shape[0]):
    local_k1[i, num1_s[i] * 60:num2_s[i] * 60] = local_k1[i, num1_s[i] * 60:num2_s[i] * 60] * stable[i, 15:16]
    local_k1[i, num2_s[i] * 60:] = local_k1[i, num2_s[i] * 60:] * stable[i, 16:17]

    local_k2[i, num1_s[i] * 60:num2_s[i] * 60] = local_k2[i, num1_s[i] * 60:num2_s[i] * 60] * stable[i, 17:18]
    local_k2[i, num2_s[i] * 60:] = local_k2[i, num2_s[i] * 60:] * stable[i, 18:19]


num1_r = torch.round(random[:, 24:25]).int()
local_k1r = torch.ones((random.shape[0], 600)).to(device)
local_k2r = torch.ones((random.shape[0], 600)).to(device)

for i in range(random.shape[0]):
    local_k1r[i, num1_r[i] * 60:] = local_k1r[i, num1_r[i] * 60:] * random[i, 25:26]
    local_k2r[i, num1_r[i] * 60:] = local_k2r[i, num1_r[i] * 60:] * random[i, 26:27]

local_k1 = local_k1.repeat_interleave(18, dim = 0)
local_k2 = local_k2.repeat_interleave(18, dim = 0)
local_k1r = local_k1r.repeat_interleave(8, dim = 0)
local_k2r = local_k2r.repeat_interleave(8, dim = 0)

local_k1 = torch.cat((local_k1, local_k1, local_k1), dim = 1)
local_k2 = torch.cat((local_k2, local_k2, local_k2), dim = 1)
local_k1r = torch.cat((local_k1r, local_k1r, local_k1r), dim = 1)
local_k2r = torch.cat((local_k2r, local_k2r, local_k2r), dim = 1)

local_k1[stablekapa == 0] = 0
local_k2[stablekapa == 0] = 0
local_k1r[randomkapa == 0] = 0
local_k2r[randomkapa == 0] = 0

In [6]:
valid_num = 10
feature_normalize11, valid_feature_normalize11 = split_data(feature_normalize11, -18 * valid_num)
feature_normalize22, valid_feature_normalize22 = split_data(feature_normalize22, -8 * valid_num)
stablekapa, valid_stablekapa = split_data(stablekapa, -18 * valid_num)
randomkapa, valid_randomkapa = split_data(randomkapa, -8 * valid_num)
stabletau, valid_stabletau = split_data(stabletau, -18 * valid_num)
randomtau, valid_randomtau = split_data(randomtau, -8 * valid_num)
stablekapaidea, valid_stablekapaidea = split_data(stablekapaidea, -18 * valid_num)
randomkapaidea, valid_randomkapaidea = split_data(randomkapaidea, -8 * valid_num)
stabletauidea, valid_stabletauidea = split_data(stabletauidea, -18 * valid_num)
randomtauidea, valid_randomtauidea = split_data(randomtauidea, -8 * valid_num)
stablealpha, valid_stablealpha = split_data(stablealpha, -18 * valid_num)
randomalpha, valid_randomalpha = split_data(randomalpha, -8 * valid_num)
stableU, valid_stableU = split_data(stableU, -18 * valid_num)
randomU, valid_randomU = split_data(randomU, -8 * valid_num)
stablew, valid_stablew = split_data(stablew, -18 * valid_num)
randomw, valid_randomw = split_data(randomw, -8 * valid_num)

local_k1, valid_local_k1 = split_data(local_k1, -18 * valid_num)
local_k2, valid_local_k2 = split_data(local_k2, -18 * valid_num)
local_k1r, valid_local_k1r = split_data(local_k1r, -8 * valid_num)
local_k2r, valid_local_k2r = split_data(local_k2r, -8 * valid_num)

In [7]:
feature_normalize11, stablekapa, stabletau, stablekapaidea, stabletauidea, stablealpha, stableU, stablew, local_k1, local_k2 = delete_row(0.01333, feature_normalize11, stablekapa, stabletau, stablekapaidea, stabletauidea, stablealpha, stableU, stablew, local_k1, local_k2)
feature_normalize22, randomkapa, randomtau, randomkapaidea, randomtauidea, randomalpha, randomU, randomw, local_k1r, local_k2r = delete_row(0.01333, feature_normalize22, randomkapa, randomtau, randomkapaidea, randomtauidea, randomalpha, randomU, randomw, local_k1r, local_k2r)

In [8]:
train_feature, test_feature, train_mask, test_mask, train_process1, test_process1, train_idea1, test_idea1, train_true1, test_true1, train_know, test_know, train_mask1, test_mask1, train_process2, test_process2, train_idea2, test_idea2, train_true2, test_true2, train_mask2, test_mask2, train_k1, test_k1, train_k2, test_k2 = DataProcess(feature_normalize11, stablekapa, stabletau, stablekapaidea, stabletauidea, stablealpha, stableU, stablew, feature_normalize22, randomkapa, randomtau, randomkapaidea, randomtauidea, randomalpha, randomU, randomw, local_k1, local_k2, local_k1r, local_k2r, seed = 1, num_sample1 = 1800, num_sample2 = 1500)

In [9]:
feature_mean = train_feature.mean(dim = 0)
feature_std = train_feature.std(dim = 0) + 1e-6

process_mean = train_process1.sum(dim = (0, 1)) / train_mask1.unsqueeze(2).sum(dim = (0, 1))
process_std = ((train_process1 - process_mean) * train_mask1.unsqueeze(2)) ** 2
process_std1 = torch.sqrt(process_std.sum(dim = (0, 1)) / train_mask1.unsqueeze(2).sum(dim = (0, 1)) + 1e-6)

idea_mean = train_idea1.sum(dim = (0, 1)) / train_mask1.unsqueeze(2).sum(dim = (0, 1))
idea_std = ((train_idea1 - idea_mean) * train_mask1.unsqueeze(2)) ** 2
idea_std1 = torch.sqrt(idea_std.sum(dim = (0, 1)) / train_mask1.unsqueeze(2).sum(dim = (0, 1)) + 1e-6)

train_feature = (train_feature - feature_mean) / feature_std
test_feature = (test_feature - feature_mean) / feature_std

train_process1 = (train_process1 - process_mean) / process_std1 * train_mask1.unsqueeze(2)
test_process1 = (test_process1 - process_mean) / process_std1 * test_mask1.unsqueeze(2)

train_idea1 = (train_idea1 - idea_mean) / idea_std1 * train_mask1.unsqueeze(2)
test_idea1 = (test_idea1 - idea_mean) / idea_std1 * test_mask1.unsqueeze(2)

train_know[:, :, 0] = train_know[:, :, 0] * 100
test_know[:, :, 0] = test_know[:, :, 0] * 100
train_know[:, :, 1] = train_know[:, :, 1] * 300
test_know[:, :, 1] = test_know[:, :, 1] * 300

train_true1[:, :, 0] = train_true1[:, :, 0] * 100
test_true1[:, :, 0] = test_true1[:, :, 0] * 100
train_true1[:, :, 1] = train_true1[:, :, 1] * 300
test_true1[:, :, 1] = test_true1[:, :, 1] * 300

In [10]:
process_mean2 = train_process2.sum(dim = (0, 1)) / train_mask2.unsqueeze(2).sum(dim = (0, 1))
process_std = ((train_process2 - process_mean2) * train_mask2.unsqueeze(2)) ** 2
process_std2 = torch.sqrt(process_std.sum(dim = (0, 1)) / train_mask2.unsqueeze(2).sum(dim = (0, 1)) + 1e-6)

idea_mean2 = train_idea2.sum(dim = (0, 1)) / train_mask2.unsqueeze(2).sum(dim = (0, 1))
idea_std = ((train_idea2 - idea_mean2) * train_mask2.unsqueeze(2)) ** 2
idea_std2 = torch.sqrt(idea_std.sum(dim = (0, 1)) / train_mask2.unsqueeze(2).sum(dim = (0, 1)) + 1e-6)

train_process2 = (train_process2 - process_mean2) / process_std2 * train_mask2.unsqueeze(2)
test_process2 = (test_process2 - process_mean2) / process_std2 * test_mask2.unsqueeze(2)

train_idea2 = (train_idea2 - idea_mean2) / idea_std2 * train_mask2.unsqueeze(2)
test_idea2 = (test_idea2 - idea_mean2) / idea_std2 * test_mask2.unsqueeze(2)

train_true2[:, :, 0] = train_true2[:, :, 0] * 100
test_true2[:, :, 0] = test_true2[:, :, 0] * 100
train_true2[:, :, 1] = train_true2[:, :, 1] * 300
test_true2[:, :, 1] = test_true2[:, :, 1] * 300

In [11]:
# GLMC-FNO
y_pred1_kapa = pd.read_csv('pred_store\\pred1_kapa11.csv', header = None).to_numpy()
y_pred1_tau = pd.read_csv('pred_store\\pred1_tau11.csv', header = None).to_numpy()
y_pred2_kapa = pd.read_csv('pred_store\\pred2_kapa11.csv', header = None).to_numpy()
y_pred2_tau = pd.read_csv('pred_store\\pred2_tau11.csv', header = None).to_numpy()

y_pred1_kapa = torch.from_numpy(y_pred1_kapa).float().to(device)
y_pred1_tau = torch.from_numpy(y_pred1_tau).float().to(device)
y_pred2_kapa = torch.from_numpy(y_pred2_kapa).float().to(device)
y_pred2_tau = torch.from_numpy(y_pred2_tau).float().to(device)

y_pred1 = torch.cat((y_pred1_kapa.unsqueeze(2), y_pred1_tau.unsqueeze(2)), dim = 2)
y_pred2 = torch.cat((y_pred2_kapa.unsqueeze(2), y_pred2_tau.unsqueeze(2)), dim = 2)

# y_pred1 = mirror_fill(y_pred1)
# y_pred2 = mirror_fill(y_pred2)

In [12]:
last_nonzero_idx1 = (train_true2 != 0).any(dim = -1).sum(dim = 1)
last_nonzero_idx2 = (test_true2 != 0).any(dim = -1).sum(dim = 1)
start_idx1 = torch.zeros_like(last_nonzero_idx1)
start_idx2 = torch.zeros_like(last_nonzero_idx2)
for i in range(last_nonzero_idx1.shape[0]):
    if last_nonzero_idx1[i] == 0:
        start_idx1[i] = 1
    else:
        start_idx1[i] = last_nonzero_idx1[i] - 120

for i in range(last_nonzero_idx2.shape[0]):
    if last_nonzero_idx2[i] == 0:
        start_idx2[i] = 1
    else:
        start_idx2[i] = last_nonzero_idx2[i] - 120
# start_idx1 = last_nonzero_idx1 - 120
# start_idx2 = last_nonzero_idx2 - 120
batch_indices1 = torch.arange(train_true1.shape[0]).to(device)
batch_indices2 = torch.arange(test_true1.shape[0]).to(device)
time_indices1 = torch.arange(120).unsqueeze(0).to(device) + start_idx1.unsqueeze(1)
time_indices2 = torch.arange(120).unsqueeze(0).to(device) + start_idx2.unsqueeze(1)

# train_true1s = train_true1[batch_indices1.unsqueeze(1), time_indices1]
# test_true1s = test_true1[batch_indices2.unsqueeze(1), time_indices2]

initial_value1 = y_pred1[batch_indices1, start_idx1 - 1]
initial_value2 = y_pred2[batch_indices2, start_idx2 - 1]

In [13]:
train_know = mirror_fill(train_know)
test_know = mirror_fill(test_know)
train_true1 = mirror_fill(train_true1)
test_true1 = mirror_fill(test_true1)
train_process1 = mirror_fill(train_process1)
test_process1 = mirror_fill(test_process1)
train_idea1 = mirror_fill(train_idea1)
test_idea1 = mirror_fill(test_idea1)
train_k1 = mirror_fill(train_k1)
test_k1 = mirror_fill(test_k1)

train_true2 = mirror_fill(train_true2)
test_true2 = mirror_fill(test_true2)
train_process2 = mirror_fill(train_process2)
test_process2 = mirror_fill(test_process2)
train_idea2 = mirror_fill(train_idea2)
test_idea2 = mirror_fill(test_idea2)
train_k2 = mirror_fill(train_k2)
test_k2 = mirror_fill(test_k2)

In [13]:
t1 = torch.linspace(0, 1, 1200).reshape(1200, 1)
t_s = t1.to(device)

train_knows = torch.cat((train_know, t_s.repeat(train_feature.shape[0], 1, 1)), dim = 2)
test_knows = torch.cat((test_know, t_s.repeat(test_feature.shape[0], 1, 1)), dim = 2)


In [14]:
x = train_feature.unsqueeze(2)
x1 = torch.cat((train_process2, train_k2, train_idea2, y_pred1, train_knows), dim = 2)
x_mask = train_mask2.unsqueeze(2)
x_maskk = (train_mask.unsqueeze(2) != train_mask2.unsqueeze(2)).int()
x1 = x1.permute(0, 2, 1)
for i in range(last_nonzero_idx1.shape[0]):
    if last_nonzero_idx1[i] == 0:
        x_maskk[i] = 0

x_train = (x, x1, x_mask, x_maskk, initial_value1, start_idx1, y_pred1)
y_train = train_true2

x = test_feature.unsqueeze(2)
x2 = torch.cat((test_process2, test_k2, test_idea2, y_pred2, test_knows), dim = 2)
x_mask2 = test_mask2.unsqueeze(2)
x_maskk2 = (test_mask.unsqueeze(2) != test_mask2.unsqueeze(2)).int()
x2 = x2.permute(0, 2, 1)
for i in range(last_nonzero_idx2.shape[0]):
    if last_nonzero_idx2[i] == 0:
        x_maskk2[i] = 0

x_test = (x, x2, x_mask2, x_maskk2, initial_value2, start_idx2, y_pred2)
y_test = test_true2

In [15]:
class CustomDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data[0])
    
    def __getitem__(self, idx):
        feature, input, mask, maskk, init, nidx, know = self.data[0], self.data[1], self.data[2], self.data[3], self.data[4], self.data[5], self.data[6]
        label1 = self.label
        
        feature1, input1, masks, maskks, init1, nidx1, know1 = feature[idx], input[idx], mask[idx], maskk[idx], init[idx], nidx[idx], know[idx]
        label11 = label1[idx]

        x = (feature1, input1, masks, maskks, init1, nidx1, know1)
        y = (label11)        

        return x, y
    
train_loader = DataLoader(CustomDataset(x_train, y_train), batch_size = 64, shuffle = True)
test_loader = DataLoader(CustomDataset(x_test, y_test), batch_size = 64, shuffle = False)

In [16]:
model = FNO(n_modes = [64], in_channels = 12, out_channels = 1, hidden_channels = 64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1.2e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 500, gamma = 0.5)

In [ ]:
steps = 1100
min_loss = 0.005
min_loss2 = 0.01
# min_loss = 2.1e-4
# min_loss2 = 3.5e-4
loss_list = []
test_loss_list = []
lossf = nn.MSELoss(reduction = 'none')
lossf1 = nn.MSELoss(reduction = 'mean')
for i in range(steps):
    model.train()
    for data, label in train_loader:
        optimizer.zero_grad()
        batch_indices = torch.arange(data[0].shape[0]).to(device)
        out = model(data[0], data[1])
        indices_sct = torch.arange(out.shape[1]).unsqueeze(0).expand(out.shape[0], out.shape[1]).to(device)
        mask_sct = indices_sct < data[5].unsqueeze(1)
        
        know = data[6][:, :, 0:1] # 预测曲率还是挠率
        # know = data[6]
        out[mask_sct] = know[mask_sct]

        label = label[:, :, 0:1] # 预测曲率还是挠率
        loss1 = lossf(out * data[3], label * data[3]).sum() / data[3].sum()
        # loss1 = lossf(out * data[3], label * data[3]).sum() / data[3].sum() / 2

        initv = data[4][:, 0:1] # 预测曲率还是挠率
        # initv = data[4]
        loss2 = lossf1(out[batch_indices, data[5]], initv)

        diff1 = (out[:, :-2] - 2 * out[:, 1:-1] + out[:, 2:]) * data[2][:, 2:] / 5
        loss3 = (diff1 ** 2).sum() / data[2][:, 2:].sum() / 2
        
        loss = loss1 + 5 * loss2 + loss3
        loss.backward(retain_graph = True)
        optimizer.step()
    loss_list.append(loss.item())
    scheduler.step()

    model.eval()
    total_test_loss = 0
    total_test_samples = 0
    with torch.no_grad():
        for data, label in test_loader:
            out = model(data[0], data[1])
            label = label[:, :, 0:1]
            test_loss = lossf(out * data[3], label * data[3]).sum() / data[3].sum()
            # test_loss = lossf(out * data[3], label * data[3]).sum() / data[3].sum() / 2

            total_test_loss += test_loss.item() * len(data[0])
            total_test_samples += len(data[0])

        average_test_loss = total_test_loss / total_test_samples
        test_loss_list.append(average_test_loss)

    if (i + 1) % 300 == 0:
        print('+++++++++++++', i + 1, '+++++++++++++')
        print('loss is: ', loss1.detach().cpu().numpy())
        print('test_loss is: ', average_test_loss)

    if loss1 < min_loss and average_test_loss < min_loss2:
        min_loss = loss1
        min_loss2 = average_test_loss
        torch.save(model.state_dict(), 'store_model\\modelVanillaFNO_4.pth')
        print('min_loss is: ', min_loss.detach().cpu().numpy(), 'loss is', average_test_loss)
